# ELT Project
Group: Jennifer Lawless, Mingming Chen, Jie Bai

In [2]:
!pip install psycopg2

In [3]:
import pandas as pd
from sqlalchemy import create_engine
import matplotlib

### Extract CSVs into DataFrames

In [4]:
csv_file = 'Resources/alldata.csv'
datascientist_locations = pd.read_csv(csv_file)
datascientist_locations.head(2)

position             company  \
0                               Development Director             ALS TDI   
1  An Ostentatiously-Excitable Principal Research...  The Hexagon Lavish   

                                         description  reviews  \
0  Development Director\nALS Therapy Development ...      NaN   
1  Job Description\n\n"The road that leads to acc...      NaN   

             location  
0  Atlanta, GA 30301   
1         Atlanta, GA

In [5]:
csv_file1 = 'Resources/indeed_job_dataset.csv'
datascientist_skills = pd.read_csv(csv_file1)
datascientist_skills.head(2)

Unnamed: 0       Job_Title  \
0           0  Data Scientist   
1           1  Data Scientist   

                                                Link Queried_Salary  \
0  https://www.indeed.com/rc/clk?jk=6a105f495c36a...         <80000   
1  https://www.indeed.com/rc/clk?jk=86afd561ea8c6...         <80000   

         Job_Type                                  Skill  Unnamed: 6  \
0  data_scientist                               SAP, SQL         NaN   
1  data_scientist  Machine Learning, R, SAS, SQL, Python         NaN   

   No_of_Skills                        Company  No_of_Reviews  ...  MD  DC NC  \
0             2                Express Scripts         3301.0  ...   0   0  0   
1             5  Money Mart Financial Services            NaN  ...   0   0  0   

  Other_states Consulting and Business Services Internet and Software  \
0            1                                0                     0   
1            0                                0                     0   

  Banks and Financial Services  Health Care  Insurance  Other_industries  
0                            0            1          0                 0  
1                            0            0          0                 0  

[2 rows x 44 columns]

### Transform location DataFrame

In [6]:
# Create a filtered dataframe from specific columns
datascientist_locations = datascientist_locations[['position','company','location']].copy()
datascientist_locations.head(2)

# Clean the data by dropping NaN rows
datascientist_locations = datascientist_locations.dropna(how='any')
# Verify the dropped rows
datascientist_locations.count()

position    6953
company     6953
location    6953
dtype: int64

### Transform Skill DataFrame 

In [17]:
# Create a filtered dataframe from specific columns
datascientist_skills = datascientist_skills[['Job_Type','Skill','Company']].copy()
datascientist_skills.head(2)

# Rename the column headers
datascientist_skills = datascientist_skills.rename(columns={"Job_Type": "job_type",
                                                         "Skill": "skill",
                                                         "Company": "company"})

# Clean the data by dropping NaN rows
datascientist_skills = datascientist_skills.dropna(how='any')
# Verify the dropped rows
datascientist_skills.count()

job_type    5384
skill       5384
company     5384
dtype: int64

### Create database connection

In [9]:
connection_string = "postgres:changeme@localhost:5432/data_job"
engine = create_engine(f'postgresql://{connection_string}')

In [10]:
# Confirm tables
engine.table_names()

['datascience_location', 'datascience_skill']

### Load DataFrames into database

In [15]:
datascientist_locations.to_sql(name='datascience_location', con=engine, if_exists='append', index=False)

In [18]:
datascientist_skills.to_sql(name='datascience_skill', con=engine, if_exists='append', index=False)

In [23]:
# Comfirm data has been added by querying the datascience_location table
pd.read_sql_query('select * from datascience_location', con=engine).head(2)

position             company  \
0                               Development Director             ALS TDI   
1  An Ostentatiously-Excitable Principal Research...  The Hexagon Lavish   

             location  
0  Atlanta, GA 30301   
1         Atlanta, GA

In [20]:
# Comfirm data has been added by querying the datascience_skill table
pd.read_sql_query('select * from datascience_skill', con=engine).head(2)

job_type                                  skill  \
0  data_scientist                               SAP, SQL   
1  data_scientist  Machine Learning, R, SAS, SQL, Python   

                         company  
0                Express Scripts  
1  Money Mart Financial Services